# Remote queue configurations

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  

### Common queue configuration

The above configurations specify on to access, synchronize files and execute commands on a remote host and are enough for it to become a remote host to execute workflows (with option `-r`). More information is needed for the host to execute external tasks, the common task-related configurations include.

#### `queue_type`

Option `query_type` determines the type of remote server or job queue. SoS currently supports the following types of job queues:

1. **`process`**: this is the default queue type. Tasks are executed directly, either on local host or on a server.
2. **`pbs`**: A PBS/MOAB/LFS/Slurm cluster system where tasks are submitted using commands such as `qsub`.
3. **`rq`**: A redis queue where tasks are submitted to the rq server and monitored through rq-dashboard.

#### `wait_for_task`

Whether or not SoS should wait for the completion of tasks (`true/false`). By default SoS will wait for the completion of tasks submitted in a queue and complete the workflow in one run. Alternatively, SoS would quit after all tasks have been submitted to a queue with `wait_for_task` set to `false`. The workflow can be resumed after tasks have been completed.

This option is set to `True` by default and can be overridden by command line option `-w` (wait) or `-W` (no wait).

#### `status_check_interval`

Option `status_check_interval` determines frequency at which SoS checks status of jobs. This is set by default to 2 seconds for `process` queue type, and `10` seconds for all other types. This number should be set to at least `60` for remote servers and longer jobs because it can be a burden to query the status of jobs very frequently.

#### `max_running_jobs`

Maximum number of running jobs. This setting controls how SoS releases tasks to job queues and is independent of possible maximum running job settings of individual task queues.

This option is set to half of the number of CPU cores by default and can be overriden by command line option `-J`.

#### `max_cores`

Maximum number of processes (an integer) allowed for the queue. A task would fail immediately if it specifies more `cores` than this option. For queues that do not require a `cores` option (e.g. a `process` queue), tasks that use more processes than `max_cores` will be automatically killed.

This option, along with `max_walltime` and `max_mem` that are described below, help prevent the submission of erraneous tasks to the queue. It also helps prevent the execution large jobs on wrong servers, such as the headnode of a cluster system on which only job preparation and submission steps are allowed.

#### `max_walltime`

Maximum walltime allowed for the queue. `max_walltime` could be specified as a string in the format of `HH:MM:SS`, or an integer with units `s` (second), `m` (minute), `h` (hour), or `d` (day). A task would fail immediately if it specifies more `walltime` than this option. For queues that do not require a `walltime` option (e.g. a `process` queue), tasks that has exceeded `max_walltime` will be automatically killed.

#### `max_mem`

Maximum memory comsumption allowed for the queue. `max_mem` could be specified as an integer (bytes) or a string with units such as `M`, `G`, `MB` etc. A task would fail immediately if it specifies more `mem` than this option. For queues that do not require a `mem` option (e.g. a `process` queue), tasks that use more RAM than `max_mem` will be automatically killed.

#### `description`

An optional short description for the queue that will be displayed with commands such as `sos status -q`.

### Process configuration

The `process` task engine is the default task engine that execute tasks without any spooling mechanism. It by default execute command `sos execute task` with appropriate verbosity level, run mode and signature mode. However, a `job_template` is allowed to create a shell script to execute the task.

#### **`job_template`**

Option `job_template` is a template of a shell script that will be executed by the process task engine. The template will be interpolated with 

* `task`: task ID
* `verbosity`, `sig_mode`, and `run_mode`: execution mode of task.
* `nodes`, `cores`, `mem`, and `walltime` as defined by corresponding task options. These are rarely useful in process engines.

So whereas a default `job_template` is equivalent to

```
    job_template: |
      sos execute {task} -v {verbosity} -s {sig_mode} \
        {'--dryrun' if run_mode == 'dryrun' else ''}
```

You can customize this script with thing like

```
    job_template: |
      export PATH=/my/additional/path:$PATH
      sos execute {task} -v {verbosity} -s {sig_mode} \
        {'--dryrun' if run_mode == 'dryrun' else ''}
```

#### **`batch_size`**

This option is related to how process queue handles multiple tasks. Simply put, if there are 100 tasks and `max_running_jobs=4`, SoS would assign `25` tasks to each job so that each job would execute `25` tasks sequentially.

Process queue has a default `batch_size` of `1000` which should work well for most cases. However, if your tasks vary greatly in runtime, there is a chance that some long-running jobs would be stacked into a single job so that this job will take much longer to execute than others. In this case you can adjust `batch_size` to a small number to submit tasks in a finer scale.

## Further reading

* 